In [29]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import re
import pickle
from tqdm.notebook import tqdm

# Import credentials
import amazon_credentials

# Connect to Amazon API
import boto3
os.environ["AWS_ACCESS_KEY_ID"] = amazon_credentials.AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = amazon_credentials.AWS_SECRET_ACCESS_KEY
os.environ["AWS_REGION"] = amazon_credentials.AWS_REGION

In [26]:
a = response["SentimentScore"]
key = max(a, key=a.get)
a[key]

0.9998409748077393

In [ ]:
max(sentiment_score, key=sentiment_score.get)

In [30]:
def split_text_into_blocks(text, headings):

    # Split the text into blocks based on the headings
    text_blocks = {}

    # Iterate over the headings
    for heading in range(len(headings)):

        if heading == 0:
            document_intro = text.split(headings[heading])[0]
            text_blocks['Document_intro'] = document_intro
        
        text_after_heading = text.split(headings[heading])[1]
        if heading == len(headings) - 1:
            text_blocks[headings[heading]] = text_after_heading
            break
        else:
            text_of_heading = text_after_heading.split(headings[heading+1])[0]
            text_blocks[headings[heading]] = text_of_heading
    
    return text_blocks

In [31]:
def clean_text_blocks(text_blocks):

    # iterate over text blocks, removing '\u200b' and extra spaces
    for key in text_blocks:
        text_blocks[key] = re.sub('\u200b', '', text_blocks[key])
        text_blocks[key] = re.sub(' +', ' ', text_blocks[key])

    return text_blocks

In [43]:
def amazon_analyze_sentiment(text):
    comprehend = boto3.client(service_name='comprehend', region_name="us-west-2")
    sentiment_response = comprehend.detect_sentiment(Text=text, LanguageCode='en')
    return sentiment_response["SentimentScore"]

In [41]:
# Load pdf text and headings from the pickle file
pdf_texts = pickle.load(open("pdf_texts.pkl", "rb"))
pdf_headings = pickle.load(open("pdf_headings.pkl", "rb"))

In [48]:
polarity_scores = {}

for pdf_name in tqdm(pdf_texts):

    try:

        if pdf_name == "FINAL-Q2-23-Shareholder-Letter" or pdf_name == "Final-Q1-23-Shareholder-Letter":
            continue

        else:

            text_blocks_scores = []

            text = pdf_texts[pdf_name]
            headings = pdf_headings[pdf_name]

            # split the text into blocks based on the headings
            text_blocks = split_text_into_blocks(text, headings)
            # clean the text blocks
            text_blocks = clean_text_blocks(text_blocks)

            # iterate over the text blocks individually, otherwise single request with all text will fail
            for heading, text in text_blocks.items():
                
                if len(text) < 5000:
                    polarity_score = amazon_analyze_sentiment(text)
                    key = max(polarity_score, key=polarity_score.get)
                    text_blocks_scores.append(polarity_score[key])
            
            polarity_scores[pdf_name] = np.mean(text_blocks_scores)
            
            break

    except Exception as e:

        print(f"Exception occurred in file {pdf_name}")
        print(f"Exception message: {str(e)}")

        break

  0%|          | 0/50 [00:00<?, ?it/s]

In [50]:
text_blocks_scores

[0.7302021980285645,
 0.9798567891120911,
 0.9805167317390442,
 0.8968285918235779,
 0.8543012738227844,
 0.9246026277542114,
 0.9749899506568909,
 0.9163967370986938,
 0.9347952008247375]

In [47]:
for heading, text in text_blocks.items():

    print(len(text))

340
2519
1161
2104
723
1193
1602
311
256
12245


In [33]:
polarity_amazon = pd.DataFrame(list(polarity_scores.items()), columns=['pdf_name', 'polarity'])
subjectivity_amazon = pd.DataFrame(list(subjectivity_scores.items()), columns=['pdf_name', 'subjectivity'])

In [35]:
polarity_google.head()

,pdf_name,polarity
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.2
1,FINAL-Q1-18-Shareholder-Letter,0.2
2,FINAL-Q1-19-Shareholder-Letter,0.2
3,FINAL-Q1-20-Shareholder-Letter,0.0
4,FINAL-Q1-21-Shareholder-Letter,0.1


In [36]:
subjectivity_google.head()

,pdf_name,subjectivity
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,37.099998
1,FINAL-Q1-18-Shareholder-Letter,27.000000
2,FINAL-Q1-19-Shareholder-Letter,23.400000
3,FINAL-Q1-20-Shareholder-Letter,59.700001
4,FINAL-Q1-21-Shareholder-Letter,23.799999
